In [1]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
from tabulate import tabulate

# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential3.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [2]:
theme_lookup = {"01": "Points", "02": "Lines", "03": "Polygons", "04": "Hexagons", "05": "Blue", "06": "Red", "07": "Green", "08": "Yellow", "09": "Monochrome", "10": "Grid", "11": "3D", "12": "Map not made with GIS software", "13": "Raster", "14": "Climate change", "15": "Connections", "16": "Island(s)", "17": "Historical map", "18": "Landuse", "19": "NULL", "20": "Population", "21": "Water", "22": "Movement", "23": "Boundaries", "24": "Elevation", "25": "COVID-19", "26": "Map with a new tool", "27": "Big or small data", "28": "Non-geographic map", "29": "Globe", "30": "A map"}

#### Set date

In [3]:
date = '2020-11-06'
start = '2020-11-06 00:00:00'
end = '2020-11-07 00:00:00'

In [93]:
# date = '2020-11-03'
# df = pd.read_csv("./data/{}.csv".format(date))
# print("{}.csv".format(date))

2020-11-03.csv


#### Collect tweets with hashtag `#30mapchallenges`

In [4]:
def collect_tweets(search_words, since):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,
                               q=search_words,
                               since= since, 
                               include_entities=True, 
                               tweet_mode="extended").items():
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.full_text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media')[0].get('media_url'))
#             print(tweet.entities.get('media'))
        except Exception as e:
            media.append(None)
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet':retweet_count,
                        'favorite':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [5]:
search_words = "#30DayMapChallenge -filter:retweets"
since = date

df = collect_tweets(search_words, since)

df = df[(df.tweet_created_at >= start) & 
   (df.tweet_created_at <= end)]

# df1 = df[df['tweet_text'].str.contains("points") | df['tweet_text'].str.contains("day 1")]

df = df[df['media'].notnull()]
df['tweet_url'] = df['tweet_text'].str.extract('(?P<url>https?://[^\s]+)', expand=False).str.strip()


df.to_csv("./data/{}.csv".format(date))

In [6]:
df

,tweet_id,tweet_text,language,tweet_created_at,retweet,favorite,hashtags,user_id,user_name,user,user_loc,user_follower,friends_count,media,tweet_url
69,1324863615522856961,Día 1 Azul / Blue\nBatimetría del Golfo de Méx...,es,2020-11-06 23:57:55,1,3,"[{'text': '30DayMapChallenge', 'indices': [72,...",756293041,El Reta,EnriqueRetamoza,,364,605,http://pbs.twimg.com/media/EmLcbBOU0AMQLT7.jpg,https://t.co/qcNPzOoHsW
70,1324863614755483649,#30DayMapChallenge \n\nDay 6: Red\n\nArchaeolo...,en,2020-11-06 23:57:55,1,7,"[{'text': '30DayMapChallenge', 'indices': [0, ...",45565875,Emil Malta ❄️,emilmalta,Nuuk,176,939,http://pbs.twimg.com/media/EmLcbzwWMAAe9Cz.jpg,https://t.co/yCcpCpETW9
71,1324862602950946819,Red kites were locally extinct in Dumfries and...,en,2020-11-06 23:53:53,8,26,"[{'text': '30DayMapChallenge', 'indices': [224...",1081900377853829120,D&G place-names,dgplacenames,,566,296,http://pbs.twimg.com/media/EmLawNgWEAANBs7.jpg,https://t.co/qs3uZUgISG
72,1324858609134657536,#30DayMapChallenge\n#Day_6\n#Red\n#Wildfires\n...,und,2020-11-06 23:38:01,2,0,"[{'text': '30DayMapChallenge', 'indices': [0, ...",605417877,Daniel Ibáñez Campos,dic1991,Barcelona,60,575,http://pbs.twimg.com/media/EmLXzItXcAAHFEW.jpg,https://t.co/FfNWKPx8fM
73,1324858327336054784,#30DayMapChallenge Red \n\nZonas dialectales d...,es,2020-11-06 23:36:54,1,6,"[{'text': '30DayMapChallenge', 'indices': [0, ...",358606269,El Diego,Kantzakata,"Fonetitlán, Distrito Federal",84,231,http://pbs.twimg.com/media/EmLV9Y1UYAATjgr.jpg,https://t.co/Si5f6P0xvv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,1324507315744038912,"Day 5, Blue. #30DayMapChallenge, cuencas y sem...",es,2020-11-06 00:22:06,2,6,"[{'text': '30DayMapChallenge', 'indices': [13,...",384907052,Leticia Luna,Letyludigital,México,1123,2447,http://pbs.twimg.com/media/EmGYDR1UYAIlq-m.jpg,https://t.co/INsceOsBfD
407,1324505281993363458,#30DayMapChallenge \n\nDay 6 Suggestion (Novem...,en,2020-11-06 00:14:02,1,0,"[{'text': '30DayMapChallenge', 'indices': [0, ...",3160198320,Dr Chris #GIS #Success,DrChrisGeoSci,"Calgary, Alberta",705,660,http://pbs.twimg.com/media/EmGWiOFXUAQS2Ry.png,https://t.co/ws8bK5YNpV
408,1324503978860638209,Day 5: blue \n\nNZ statistical fisheries zones...,en,2020-11-06 00:08:51,1,3,"[{'text': '30DayMapChallenge', 'indices': [142...",12306,Luis Apiolaza 🧮🍃📷⏚,zentree,"Ōtautahi, Aotearoa",1279,282,http://pbs.twimg.com/media/EmGVUAOU8AAuhTy.jpg,https://t.co/FYSSFptaa2
409,1324502843366277121,#30DayMapChallenge - we invite you to come to ...,en,2020-11-06 00:04:20,1,0,"[{'text': '30DayMapChallenge', 'indices': [0, ...",10365542,GreenMap,GreenMap,NYC,4572,3729,http://pbs.twimg.com/media/EmGT6_2XUAESLX6.jpg,https://t.co/Cp77iHUii1


#### late entry (only for day 2)

In [7]:
# df[df['tweet_text'].str.contains('points', case=False) & 
#    df['tweet_text'].str.contains('day 1', case=False)].to_csv("./data/{}_late.csv".format(date))

In [8]:
# df = df.drop(df[df['tweet_text'].str.contains('points', case=False) & 
#                 df['tweet_text'].str.contains('day 1', case=False)].index)

---

### Overview

In [7]:
# df[['retweet','favorite', 'user_follower', 'friends_count']].describe()

#### Top favorited/retweeted `#30mapchallenges` post

In [20]:
def sort_map(df, sortby, num):
    df = df.sort_values(by=sortby, ascending=False).head(num)
    df['user'] = '**[@' + df['user'] +']('+ df['tweet_url']+')**'
    df['media'] = '![](' + df['media'] +')'
    df = df[['user', 'retweet', 'favorite', 'media']].set_index("user")
    return(tabulate(df, headers='keys', tablefmt='github'))

In [17]:
def tweet_shortcode(df, sortby, num):
    df = df.sort_values(by=sortby, ascending=False).head(num)
    df['shortcode'] = '{{< tweet ' + df['tweet_id'].apply(str) + ' >}}'
    df = df[['shortcode']]
    shortcode_ls = []
    for i in df['shortcode']:
        shortcode_ls.append(i)
    return shortcode_ls

In [21]:
# create md
with open('day{}_{}.md'.format(date[-2:], theme_lookup[date[-2:]]), 'w') as f:
    print(date +'...\n')
    print("Creating mardowkdown file ...")
    heading1 = "#### Top favorited post for day {} `#30mapchallenges ({})`".format(date[-2:], theme_lookup[date[-2:]])
    f.write(heading1+ '\n')
    f.write(sort_map(df, 'favorite', 15)+ '\n \n')
    
#     heading2 = "#### Top retweeted post for day {} `#30mapchallenges ({})`".format(date[-2:], theme_lookup[date[-2:]])
#     f.write(heading2+ '\n')
#     f.write(sort_map(df, 'retweet', 15))

2020-11-06...

Creating mardowkdown file ...


In [19]:
# create md_index
with open('../../Blog/mywebsite/content/en/post/30daymap-{}-{}.md'.format(date[-2:], theme_lookup[date[-2:]]), 'w') as f:
    print(date +'...\n')
    print("Creating mardowkdown file ...")

    heading1 = "### Top favorited post for day {} `#30mapchallenges ({})`".format(date[-2:], theme_lookup[date[-2:]])
    f.write(heading1+ '\n')
    for s in tweet_shortcode(df, 'favorite', 15):
        f.write(s + '\n')
    
    f.write('\n')

    heading2 = "### Top retweeted post for day {} `#30mapchallenges ({})`".format(date[-2:], theme_lookup[date[-2:]])
    f.write(heading2+ '\n')
    for s in tweet_shortcode(df, 'retweet', 15):
        f.write(s + '\n')

2020-11-06...

Creating mardowkdown file ...


#### Late entry

In [25]:
# df_late = pd.read_csv("./data/{}_late.csv".format(date))
# print("{}_late.csv".format(date))
# print("Add to day 1...")

2020-11-02_late.csv
Add to day 1...


In [27]:
# df_late_fav = df_late.sort_values(by='favorite', ascending=False).head(5)
# df_late_fav = df_late_fav[['user', 'retweet', 'favorite', 'media']]
# df_late_fav['user'] = '**[@' + df_late['user'] +']('+ df_late['tweet_url']+')**'
# df_late_fav['media'] = '![](' + df_late['media'] +')'

In [28]:
print(tabulate(df_late_fav, headers='keys', tablefmt='github'))

|    | user                                            |   retweet |   favorite | media                                               |
|----|-------------------------------------------------|-----------|------------|-----------------------------------------------------|
| 42 | **[@darenw](https://t.co/rZ2gtbJmI4)**          |         4 |         95 | ![](http://pbs.twimg.com/media/Elyc4d4XUAEs09h.jpg) |
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |        22 |         60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 53 | **[@PratapVardhan](https://t.co/WTT8WM17qy)**   |         4 |         43 | ![](http://pbs.twimg.com/media/Elx5722WkAEOOMf.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |         5 |         35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 54 | **[@BZgeo](https://t.co/LavH2067O2)**           |         3 |         26 | ![](http://pbs.twimg.com/media/Elx0-fPXIAc9JGJ.png) |


In [29]:
# df_late_rt = df_late.sort_values(by='retweet', ascending=False).head(5)
# df_late_rt = df_late_rt[['user', 'retweet', 'favorite', 'media']]
# df_late_rt['user'] = '**[@' + df_late['user'] +']('+ df_late['tweet_url']+')**'
# df_late_rt['media'] = '![](' + df_late['media'] +')'

In [30]:
# print(tabulate(df_late_rt, headers='keys', tablefmt='github'))

|    | user                                            |   retweet |   favorite | media                                               |
|----|-------------------------------------------------|-----------|------------|-----------------------------------------------------|
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |        22 |         60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 24 | **[@neocartocnrs](https://t.co/1GcTGraT9T)**    |         8 |         14 | ![](http://pbs.twimg.com/media/Elz2OvNXEAAItfe.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |         5 |         35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 57 | **[@nurussadad](https://t.co/z4HaIfQMBi)**      |         4 |          7 | ![](http://pbs.twimg.com/media/ElxuzgpU0AEIbdT.png) |
|  7 | **[@geologyjesse](https://t.co/dR68mE4zhD)**    |         4 |         14 | ![](http://pbs.twimg.com/media/El1vroKWoAEozEk.jpg) |


### Send tweet via API

In [48]:
status = 'A github repo of monitoring tweets feeds with wonderful maps. Github: https://github.com/hn303/30DayMapChallenge-Bot'

In [45]:
imagePath = "screenshot/github.com_hn303_30DayMapChallenge-Bot.png"

In [49]:
api.update_status(status)

Status(_api=<tweepy.api.API object at 0x000001D700B3FC88>, _json={'created_at': 'Wed Nov 04 13:14:23 +0000 2020', 'id': 1323976890621792256, 'id_str': '1323976890621792256', 'text': 'A github repo of monitoring tweets feeds with wonderful maps. **Github**:https://t.co/J1NHDOPpdT', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/J1NHDOPpdT', 'expanded_url': 'https://github.com/hn303/30DayMapChallenge-Bot', 'display_url': 'github.com/hn303/30DayMap…', 'indices': [73, 96]}]}, 'source': '<a href="https://www.landecon.cam.ac.uk/research/research-centres/lisa/members" rel="nofollow">Untitle_APP3</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1078706606488068096, 'id_str': '1078706606488068096', 'name': 'Stan', 'screen_name': 'Stan_CamLE', 'location': '', 'description': '', 'url': None, 'entit

In [46]:
api.update_with_media(imagePath, status)

Status(_api=<tweepy.api.API object at 0x000001D700B3FC88>, _json={'created_at': 'Wed Nov 04 12:58:56 +0000 2020', 'id': 1323973002543976450, 'id_str': '1323973002543976450', 'text': 'A github repo of monitoring tweets feeds with wonderful maps. https://t.co/J1NHDOPpdT https://t.co/HwA2aVTax1', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/J1NHDOPpdT', 'expanded_url': 'https://github.com/hn303/30DayMapChallenge-Bot', 'display_url': 'github.com/hn303/30DayMap…', 'indices': [62, 85]}], 'media': [{'id': 1323972999717003266, 'id_str': '1323972999717003266', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/El-ybbRXIAICiB1.jpg', 'media_url_https': 'https://pbs.twimg.com/media/El-ybbRXIAICiB1.jpg', 'url': 'https://t.co/HwA2aVTax1', 'display_url': 'pic.twitter.com/HwA2aVTax1', 'expanded_url': 'https://twitter.com/Stan_CamLE/status/1323973002543976450/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 